<p style="text-align:center;">
<img src="https://github.com/digital-futures-academy/DataScienceMasterResources/blob/main/Resources/datascience-notebook-header.png?raw=true"
     alt="DigitalFuturesLogo"
     style="float: center; margin-right: 10px;" />
</p>

### Modelling

In [182]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

import statsmodels.api as sm
import statsmodels.tools.eval_measures as sm_tools

#### Load and Check Data

In [183]:
# read data frame
df = pd.read_csv('reed_data_eda.csv')

In [184]:
# check top of df
df.head(1)

,title,company,location,contract,remote,salary,hourly_rate,day_rate,min_salary,max_salary,avg_salary,city,title_simplified,seniority,permanent,temporary,remote_binary,recruitment_consultancy
0,data scientist,eg group,guide,"permanent, full-time",no,"£30,000 per annum",0,0,30000,30000,30000,other cities,data scientist,other,1,0,0,0


In [185]:
# select columns for modelling
df = df.iloc[:,6:]
df.head()

,hourly_rate,day_rate,min_salary,max_salary,avg_salary,city,title_simplified,seniority,permanent,temporary,remote_binary,recruitment_consultancy
0,0,0,30000,30000,30000,other cities,data scientist,other,1,0,0,0
1,0,0,50000,65000,57500,other cities,data scientist,other,1,0,0,0
2,0,0,40000,70000,55000,london,data scientist,other,1,0,0,0
3,0,0,40000,50000,45000,other cities,data scientist,other,1,0,0,0
4,0,1,117000,130000,123500,other cities,data scientist,other,0,1,0,0


In [186]:
# check dimensions
df.shape

(833, 12)

#### Feature Engineering

In [187]:
# set option to view all columns
pd.set_option('display.max_columns', None)

In [188]:
# one hot encode categorical columns
df = pd.get_dummies(df, drop_first=True)
df.head()

,hourly_rate,day_rate,min_salary,max_salary,avg_salary,permanent,temporary,remote_binary,recruitment_consultancy,city_leeds,city_london,city_manchester,city_other cities,title_simplified_data analyst,title_simplified_data engineer,title_simplified_data scientist,title_simplified_machine learning engineer,title_simplified_other,seniority_other,seniority_senior
0,0,0,30000,30000,30000,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0
1,0,0,50000,65000,57500,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0
2,0,0,40000,70000,55000,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0
3,0,0,40000,50000,45000,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0
4,0,1,117000,130000,123500,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0


In [189]:
# check dimesions
df.shape

(833, 20)

#### Train, Test, Split

In [190]:
# creating X and y
X = df.drop('avg_salary', axis=1)
y = df['avg_salary']

In [191]:
# split data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 21)

In [192]:
# check train data dimensions
X_train.shape

(666, 19)

In [193]:
# check index between X_train and y_train match
(X_train.index == y_train.index).sum()

666

#### Linear Regression Model (stats models)

##### Train Model

In [194]:
# create a function to add the constant
def feature_eng(X):
    X = sm.add_constant(X)
    return X

In [195]:
# add constant to X_train
X_train_fe = feature_eng(X_train)

In [196]:
# feature columns to select for training
feature_columns =  ['const',
                    'hourly_rate',
                    'day_rate',
                    #'min_salary',
                    #'max_salary',
                    #'permanent',
                    'temporary',
                    'remote_binary',
                    'recruitment_consultancy',
                    'city_leeds',
                    'city_london',
                    'city_manchester',
                    'city_other cities',
                    'title_simplified_data analyst',
                    'title_simplified_data engineer',
                    'title_simplified_data scientist',
                    'title_simplified_machine learning engineer',
                    'title_simplified_other',
                    'seniority_other',
                    'seniority_senior']

In [197]:
# fit the model using the training data
model = sm.OLS(y_train, X_train_fe[feature_columns]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             avg_salary   R-squared:                       0.680
Model:                            OLS   Adj. R-squared:                  0.672
Method:                 Least Squares   F-statistic:                     86.11
Date:                Mon, 18 Sep 2023   Prob (F-statistic):          1.81e-148
Time:                        20:47:34   Log-Likelihood:                -7481.0
No. Observations:                 666   AIC:                         1.500e+04
Df Residuals:                     649   BIC:                         1.507e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==============================================================================================================
                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
const                                       2.366e+04   5109.798      4.631      0.000    1.36e+04    3.37e+04
hourly_rate                                 6632.1830   5412.009      1.225      0.221   -3994.978    1.73e+04
day_rate                                    6.995e+04   3908.006     17.900      0.000    6.23e+04    7.76e+04
temporary                                  -3276.3188   3544.744     -0.924      0.356   -1.02e+04    3684.233
remote_binary                                989.3772   2442.581      0.405      0.686   -3806.939    5785.693
recruitment_consultancy                     2062.2395   1646.886      1.252      0.211   -1171.628    5296.107
city_leeds                                  4619.1517   5724.634      0.807      0.420   -6621.889    1.59e+04
city_london                                 1.368e+04   4267.476      3.206      0.001    5302.622    2.21e+04
city_manchester                             4385.2542   4925.568      0.890      0.374   -5286.719    1.41e+04
city_other cities                           4131.0871   4248.970      0.972      0.331   -4212.302    1.25e+04
title_simplified_data analyst              -8758.2576   2155.774     -4.063      0.000    -1.3e+04   -4525.123
title_simplified_data engineer              5777.6213   2318.565      2.492      0.013    1224.828    1.03e+04
title_simplified_data scientist             1.215e+04   2791.950      4.351      0.000    6666.678    1.76e+04
title_simplified_machine learning engineer  2.729e+04   4641.210      5.879      0.000    1.82e+04    3.64e+04
title_simplified_other                     -5781.6994   3318.515     -1.742      0.082   -1.23e+04     734.622
seniority_other                             2.126e+04   2884.205      7.373      0.000    1.56e+04    2.69e+04
seniority_senior                            3.715e+04   3112.009     11.939      0.000     3.1e+04    4.33e+04
==============================================================================
Omnibus:                       73.463   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              353.863
Skew:                           0.353   Prob(JB):                     1.44e-77
Kurtosis:                       6.500   Cond. No.                         20.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [198]:
# predict on training data with selected feature columns
train_predictions = model.predict(X_train_fe[feature_columns])

# compare errors between y_train and train_prediction
rmse = sm_tools.rmse(y_train, train_predictions)
mae = metrics.mean_absolute_error(y_train, train_predictions)

# output results
print(f'rmse: {rmse}')
print(f'mae: {mae}')

rmse: 18285.528694978697
mae: 13205.955280508415


##### Test Model

In [199]:
# add constast to X_test
X_test_fe = feature_eng(X_test)

In [200]:
# predict on test data with selected feature columns
test_predictions = model.predict(X_test_fe[feature_columns])

# compare errors between y_test and test_prediction
rmse = sm_tools.rmse(y_test, test_predictions)
mae = metrics.mean_absolute_error(y_test, test_predictions)

# output results
print(f'rmse: {rmse}')
print(f'mae: {mae}')

rmse: 18878.369685618127
mae: 13921.636771645053


#### Ridge and Lasso Regression Models (sklearn)

##### Hyperparameter Tuning

In [223]:
# feature columns to select for training
feature_columns =  [
                    'hourly_rate',
                    'day_rate',
                    #'min_salary',
                    #'max_salary',
                    #'permanent',
                    'temporary',
                    'remote_binary',
                    'recruitment_consultancy',
                    'city_leeds',
                    'city_london',
                    'city_manchester',
                    'city_other cities',
                    'title_simplified_data analyst',
                    'title_simplified_data engineer',
                    'title_simplified_data scientist',
                    'title_simplified_machine learning engineer',
                    'title_simplified_other',
                    'seniority_other',
                    'seniority_senior'
                    ]

In [209]:
# create a function to find the best alpha value for ridge and lasso models respectively
def hyperparameter_tuning(model, alpha_range, X, y, cv=10):
    
    alpha_values = []
    errors = []

    for alpha in alpha_range:
        model_instance = model(alpha=alpha)
        scorer = metrics.make_scorer(metrics.mean_absolute_error)
        error = np.mean(cross_val_score(model_instance, X, y, scoring=scorer, cv=cv))
        alpha_values.append(alpha)
        errors.append(error)

    return alpha_values, errors

# alpha range to iterate through
alpha_range = np.arange(0.01, 1.0, 0.01)

# perform hyperparameter tuning and cross-validation for lasso
lasso_alpha_values, lasso_errors = hyperparameter_tuning(Lasso, alpha_range, X_train[feature_columns], y_train)

# perform hyperparameter tuning and cross-validation for ridge
ridge_alpha_values, ridge_errors = hyperparameter_tuning(Ridge, alpha_range, X_train[feature_columns], y_train)

# create a data frame for each model mapping alpha values to corresponding errors
df_lasso_err = pd.DataFrame({'alpha': lasso_alpha_values, 'error': lasso_errors})
df_ridge_err = pd.DataFrame({'alpha': ridge_alpha_values, 'error': ridge_errors})

# find the the alpha values with the smallest errors for each model
best_alpha_lasso = df_lasso_err[df_lasso_err['error']==min(df_lasso_err.error)]
best_alpha_ridge = df_ridge_err[df_ridge_err['error']==min(df_ridge_err.error)]

# output results
print('Best Lasso Alpha:')
print(best_alpha_lasso)
print('\n')
print('Best Ridge Alpha:')
print(best_alpha_ridge)

Best Lasso Alpha:
   alpha        error
0   0.01  13572.21331


Best Ridge Alpha:
   alpha         error
0   0.01  13572.884061


##### Train and Test Model

In [210]:
# create ridge and lasso regression models
ridge_model = Ridge(alpha=0.01) 
lasso_model = Lasso(alpha=0.01)

# fit the models to the training data
ridge_model.fit(X_train[feature_columns], y_train)
lasso_model.fit(X_train[feature_columns], y_train)

# cross-validation scores
scorer = metrics.make_scorer(metrics.mean_absolute_error)
ridge_train_score = np.mean(cross_val_score(ridge_model, X_train[feature_columns], y_train, scoring=scorer, cv=10))
lasso_train_score = np.mean(cross_val_score(lasso_model, X_train[feature_columns], y_train, scoring=scorer, cv=10))

# predict on test
ridge_predictions = ridge_model.predict(X_test[feature_columns])
lasso_predictions = lasso_model.predict(X_test[feature_columns])

# calculate mae for ridge and lasso
ridge_test_mae = metrics.mean_absolute_error(y_test, ridge_predictions)
lasso_test_mae = metrics.mean_absolute_error(y_test, lasso_predictions)

# output the results
print('Train Data:')
print(f"Ridge MAE: {ridge_train_score}")
print(f"Lasso MAE: {lasso_train_score}")
print('\n')
print('Test Data:')
print(f"Ridge MAE: {ridge_test_mae}")
print(f"Lasso MAE: {lasso_test_mae}")

Train Data:
Ridge MAE: 13572.884060582084
Lasso MAE: 13572.213310216466


Test Data:
Ridge MAE: 13921.705934996418
Lasso MAE: 13921.624636429133


#### Random Forest and Decision Tree Models

##### Hyperparameter Tuning

In [226]:
# feature columns to select for training
feature_columns =  [
                    'hourly_rate',
                    'day_rate',
                    #'min_salary',
                    #'max_salary',
                    'permanent',
                    #'temporary',
                    'remote_binary',
                    'recruitment_consultancy',
                    'city_leeds',
                    'city_london',
                    'city_manchester',
                    'city_other cities',
                    'title_simplified_data analyst',
                    'title_simplified_data engineer',
                    'title_simplified_data scientist',
                    'title_simplified_machine learning engineer',
                    'title_simplified_other',
                    'seniority_other',
                    'seniority_senior'
                    ]

In [205]:
# compare cross validation scores between decisiontree and randomforest models
dt = DecisionTreeRegressor(random_state=21)
rf = RandomForestRegressor(random_state=21)

scorer = metrics.make_scorer(metrics.mean_absolute_error)
dt_score = np.mean(cross_val_score(dt, X_train[feature_columns], y_train, scoring=scorer, cv=10))

rf_score = np.mean(cross_val_score(rf, X_train[feature_columns], y_train, scoring=scorer, cv=10))


print(f'DecisionTree: {dt_score}')

print(f'RandomForest: {rf_score}')

DecisionTree: 14847.404261096362
RandomForest: 14253.750068453533


In [206]:
# create the Random Forest regressor
rf = RandomForestRegressor(random_state=21)

# define the parameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [5 ,10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# create scorer metric for gridsearchcv
scorer = metrics.make_scorer(metrics.mean_absolute_error)

# create gridsearchcv object
gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring=scorer, cv=10, n_jobs=-1)

# fit the grid search to data
gs.fit(X_train[feature_columns], y_train)

gs.best_params_

{'max_depth': 20,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

##### Train and Test Model

In [216]:
# get the best estimator (model) with the tuned hyperparameters
best_rf_model = gs.best_estimator_

# get best score (mae)
best_rf_score = gs.best_score_

# make predictions on the test set
rf_predicitons = best_rf_model.predict(X_test[feature_columns])

rf_test_mae = metrics.mean_absolute_error(y_test, rf_predicitons)

# compare r-squared scores between train and test data
train_score = best_rf_model.score(X_train[feature_columns], y_train)
test_score = best_rf_model.score(X_test[feature_columns], y_test)

# output results
print('Train Data:')
print(f'R2: {train_score}')
print(f'MAE: {best_rf_score}')
print('\n')
print('Test Data:')
print(f'R2: {test_score}')
print(f'MAE: {rf_test_mae}')

Train Data:
R2: 0.7806242244063012
MAE: 14253.750068453533


Test Data:
R2: 0.71278436923419
MAE: 14198.89591936066


#### Combined Model

In [227]:
combined_model = metrics.mean_absolute_error(y_test, (rf_predictions + lasso_predictions)/2)

print(f'MAE: {combined_model}')

MAE: 13511.580841252511
